# Dependencies

In [1]:
import pandas as pd
import numpy as np
from musixmatch import Musixmatch
from API_Key import api_key2
import time
from pprint import pprint

In [2]:
import sqlalchemy
from sqlalchemy import create_engine, inspect, Column, Integer, Float, String, Sequence, Boolean, text
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base

In [3]:
musixmatch = Musixmatch(api_key2)

# Setting up SQLlite

In [3]:
#Using declarative base
Base = declarative_base()
engine = create_engine('sqlite:///lyrics3.db', echo = False)

In [5]:
#defining the Tracks table
class Tracks(Base):
    __tablename__ = "tracks"
    
    id = Column(Integer, primary_key=True, nullable = False, unique = True, autoincrement = True)
    album_id = Column(Integer)
    artist_id = Column(Integer)
    artist_name = Column(String)
    explicit = Column(Integer)
    has_lyrics = Column(Integer)
    lyrics_id = Column(Integer)
    track_id = Column(Integer, unique = True)
    track_name = Column(String)

In [6]:
#defining the Lyrics table
class Lyrics(Base):
    __tablename__ = "lyrics"
    
    id = Column(Integer, primary_key=True, nullable = False, unique = True, autoincrement = True)
    explicit = Column(Integer)
    lyrics_body = Column(String)
    lyrics_id = Column(Integer, unique = True)
    lyrics_language = Column(String)
    lyrics_language_description = Column(String)

In [7]:
#defining the Artists table
class Artists(Base):
    __tablename__ = "artists"
    
    id = Column(Integer, primary_key=True, nullable = False, unique = True, autoincrement = True)
    artist_id = Column(Integer, unique = True)
    artist_name = Column(String)
    artist_q = Column(String)

In [8]:
#defining the Albums table
class Albums(Base):
    __tablename__ = "albums"
    
    id = Column(Integer, primary_key=True, nullable = False, unique = True, autoincrement = True)
    artist_id = Column(Integer)
    artist_name = Column(String)
    album_id = Column(Integer, unique = True)
    album_name = Column(String)

In [5]:
#defining the genres table
class Album_Genres(Base):
    __tablename__ = "genres"
    
    id = Column(Integer, primary_key=True, nullable = False, unique = True, autoincrement = True)
    album_id = Column(String)
    album_name = Column(String)
    artist = Column(String)
    genre = Column(String)

In [6]:
#Creating the table metadata
Base.metadata.create_all(engine)
session = Session(engine)

In [7]:
#Binding engine to the metadata
meta = sqlalchemy.MetaData()
meta.reflect(bind = engine)

# MusixMatch API - Finding artists via text search from manually created list

In [11]:
artists_path = 'artist_names.xlsx'
artists_df = pd.read_excel(artists_path, sheetname = "Sheet1", header = None)
artists = list(artists_df[0].values)

In [12]:
#Connecting to tracks table
table = sqlalchemy.Table('artists', meta, autoload = True)

In [ ]:
for artist in artists:
    print(f"getting artist names {artist}")
    result = musixmatch.artist_search(q_artist = artist, page = 1, page_size = 100, f_artist_id = '', f_artist_mbid = '')
    print('extracting results')
    artist_list = result['message']['body']['artist_list']
    artists = [artist['artist'] for artist in artist_list]
    print("creating package")
    for item in artists:
        package = {
            'artist_id': item['artist_id'],
            'artist_name': item['artist_name'],
            'artist_q': artist
        }
        try:
            print("writing package to database")
            session.execute(table.insert(), package)
        except:
            next
    print("committing to database")
    session.commit()

# MusixMatch API - Finding Albums from Artists

In [26]:
artists = pd.read_sql('artists', engine)
artists.dropna(inplace = True)
artist_ids = list(artists['artist_id'].values)

In [12]:
table = sqlalchemy.Table('albums', meta, autoload = True)

In [ ]:
for artist_id in artist_ids:
    print(f"getting albums for artist {artist_id}")
    result = musixmatch.artist_albums_get(artist_id = artist_id, g_album_name = '', page = 1, page_size = 100, s_release_date = '')
    print("extracting the album_list")
    album_list = result['message']['body']['album_list']
    print("extracting individual albums")
    albums = [album['album'] for album in album_list]
    print("iterating through the list of albums")
    for item in albums:
        print("creating the package")
        package = {
            'album_id': item['album_id'],
            'album_name': item['album_name'],
            'artist_id': item['artist_id'],
            'artist_name': item['artist_name']
        }
        try:
            print("writing to the database")
            session.execute(table.insert(), package)
        except:
            next
    print("comitting to the database")
    session.commit()

# MusixMatch API - Scraping Tracks From Charts

In [19]:
#Connecting to tracks table
table = sqlalchemy.Table('tracks', meta, autoload = True)

In [10]:
country_codes = ['AU', 'CA', 'IE', 'IM', 'NZ', 'GB', 'US']

In [ ]:
country_count = 1
for country in country_codes:
    
    print(f"Attempting country {country_count} for {country}")
    
    page_count = 1

    while page_count < 5:
        print(f"calling api attempt {page_count} for {country}")
        result = musixmatch.chart_tracks_get(page_count, 100, f_has_lyrics = True)
        tracks = result['message']['body']['track_list']
        track_list = [track['track'] for track in tracks]
        print(f"track list is {len(track_list)}")
        for track in track_list:
            package = {
                'album_id': track['album_id'],
                'artist_id': track['artist_id'],
                'artist_name': track['artist_name'],
                'explicit': track['explicit'],
                'has_lyrics': track['has_lyrics'],
                'lyrics_id': track['lyrics_id'],
                'track_id': track['track_id'],
                'track_name': track['track_name']
            }
            print(package['track_name'])
            print("writing to database")
            try:
                session.execute(table.insert(), package)
            except:
                next
        page_count += 1
        time.sleep(1)
        session.commit()
    
    country_count += 1

# MusixMatch API - finding tracks from albums

## Starting point is the tracks data from the charts

In [20]:
tracks = pd.read_sql_table('tracks', engine, index_col = "id", coerce_float = False)

In [21]:
album_ids = tracks['album_id'].values

In [24]:
#Connecting to tracks table
table = sqlalchemy.Table('tracks', meta, autoload = True)

In [ ]:
album_count = 1

for album_id in album_ids:
    print(f"getting result {album_count} for {album_id}")
    result = musixmatch.album_tracks_get(album_id = album_id, album_mbid = "", page = 1, page_size = 100)
    print(f"getting track_list for {album_id}")
    album = result['message']['body']['track_list']
    print("creating track list")
    track_list = [track['track'] for track in album]
    track_count = 1
    print(f"Iterating through tracks in album")
    for track in track_list:
        print(f"track is {track['track_id']}")
        package = {
                'album_id': int(track['album_id']),
                'artist_id': int(track['artist_id']),
                'artist_name': track['artist_name'],
                'explicit': int(track['explicit']),
                'has_lyrics': int(track['has_lyrics']),
                'lyrics_id': int(track['lyrics_id']),
                'track_id': int(track['track_id']),
                'track_name': track['track_name']
            }
        print(package['track_name'])
        print("writing to database")
        try:
            session.execute(table.insert(), package)
        except:
            print("writing to database failed")
            next
        print("committing files to database")
        session.commit()
    album_count += 1

## Starting point is the album's data

In [15]:
albums = pd.read_sql_table('albums', engine, index_col = "id", coerce_float = False)
albums = albums.iloc[647:]
album_ids = list(albums['album_id'].values)

In [16]:
#Connecting to tracks table
table = sqlalchemy.Table('tracks', meta, autoload = True)

In [ ]:
album_count = 1

for album_id in album_ids:
    print(f"getting result {album_count} for {album_id}")
    result = musixmatch.album_tracks_get(album_id = album_id, album_mbid = "", page = 1, page_size = 100)
    print(f"getting track_list for {album_id}")
    album = result['message']['body']['track_list']
    print("creating track list")
    track_list = [track['track'] for track in album]
    track_count = 1
    print(f"Iterating through tracks in album")
    for track in track_list:
        print(f"track is {track['track_id']}")
        package = {
                'album_id': int(track['album_id']),
                'artist_id': int(track['artist_id']),
                'artist_name': track['artist_name'],
                'explicit': int(track['explicit']),
                'has_lyrics': int(track['has_lyrics']),
                'lyrics_id': int(track['lyrics_id']),
                'track_id': int(track['track_id']),
                'track_name': track['track_name']
            }
        print(package['track_name'])
        print("writing to database")
        try:
            session.execute(table.insert(), package)
        except:
            print("writing to database failed")
            next
        print("committing files to database")
        session.commit()
    album_count += 1

## MusixMatch API - Scraping Lyrics from the Albums Data

In [1]:
#Connecting to lyrics table
table = sqlalchemy.Table('lyrics', meta, autoload = True)

NameError: name 'sqlalchemy' is not defined

In [29]:
tracks = pd.read_sql('tracks', engine, index_col = "id")
track_ids = tracks['track_id'].values

In [ ]:
count = 1
for track in track_ids:
    print(f"calling API attempt {count} for track_id# {track}")
    print(f"track data type is {type(track)}")
    lyrics = musixmatch.track_lyrics_get(track)
    print("pulling out lyrics package")
    lyrics_package = lyrics['message']['body']['lyrics']
    print("pulling out package to insert into SQL database")
    package = {
        "explicit": lyrics_package['explicit'],
        "lyrics_body": lyrics_package["lyrics_body"][:-69],
        "lyrics_id": lyrics_package['lyrics_id'],
        "lyrics_language": lyrics_package['lyrics_language'],
        "lyrics_language_description": lyrics_package['lyrics_language_description'],
        "updated_time": lyrics_package['updated_time']
    }
    
    print(package['lyrics_body'][:10])
    print("writing to database")
    try:
        session.execute(table.insert(), package)
    except:
        next
    print("committing files to database")
    session.commit()
    count += 1 

# Acquiring Genre from Album

In [10]:
#Connecting to lyrics table
table = sqlalchemy.Table('album_genres', meta, autoload = True)

In [11]:
albums = pd.read_sql_table('tracks', engine, index_col = "id", coerce_float = False)
album_ids = list(albums['album_id'].values)

In [12]:
album_arr = np.array(album_ids)
album_unique = np.unique(album_arr)
print(len(album_unique))

3398


In [35]:
genre_dic = {}
for id in album_unique:
    
    result = musixmatch.album_get(album_id=id)
    album = result['message']['body']['album']['album_name']
    artist = result['message']['body']['album']['artist_name']
    print(f'{artist}: {album}')
    print('-----------------------')
    genre_dic[f'{id}'] = {"artist":artist,"album":album}

Will Smith: Big Willie Style
-----------------------
U2: War
-----------------------
Led Zeppelin: Led Zeppelin
-----------------------
Rush: Power Windows
-----------------------
Will Smith: Willennium
-----------------------
Coldplay: Parachutes
-----------------------
Rush: Hold Your Fire
-----------------------
Rush: Counterparts
-----------------------
Otis Rush: 1956-1958: Cobra Recordings
-----------------------
Otis Rush: The Cobra Sessions 1956-1958
-----------------------
Slayer: Hell Awaits
-----------------------
Slayer: Show No Mercy
-----------------------
Metallica: ...And Justice for All
-----------------------
Eagles: Eagles Greatest Hits, Volume 2
-----------------------
U2: Rattle and Hum
-----------------------
Led Zeppelin: Houses of the Holy
-----------------------
Led Zeppelin: Led Zeppelin III
-----------------------
Metallica: Metallica
-----------------------
Metallica: Reload
-----------------------
Slayer: Divine Intervention
-----------------------
Metallic

Prince Paul: Psychoanalysis: What Is It?
-----------------------
Metallica: Load
-----------------------
DJ Jazzy Jeff & The Fresh Prince: And In This Corner...
-----------------------
Metallica: St. Anger
-----------------------
Metallica: St. Anger
-----------------------
Darude: Music
-----------------------
Jennifer Rush: The Power of Jennifer Rush
-----------------------
Prince Buster: Judge Dread Rock Steady / She Was a Rough Rider
-----------------------
Prince Paul: Politics of the Business
-----------------------
Beyoncé: Dangerously In Love
-----------------------
Beyoncé: Dangerously in Love
-----------------------
Vitamin String Quartet: The String Quartet Tribute To Coldplay
-----------------------
Darude: Before the Storm
-----------------------
DJ Green Lantern feat. Eminem: Shady Records Mixtape
-----------------------
U2: Pride (In the Name of Love)
-----------------------
Slayer: Seasons in the Abyss / Reign in Blood
-----------------------
Beyoncé: Dangerously in Lov

Will Smith: Switch
-----------------------
U2: Rattle and Hum
-----------------------
Darude: Feel the Beat
-----------------------
U2: The Everlasting Love (New and Best)
-----------------------
Darude: Sandstorm
-----------------------
Bonnie "Prince" Billy: No More Workhorse Blues
-----------------------
Prince: The Hits/The B-Sides
-----------------------
Will Smith: Lost and Found
-----------------------
Will Smith: Switch
-----------------------
Otis Rush: Any Place I'm Going
-----------------------
Weezer: Make Believe
-----------------------
Tom Rush: Trolling for Owls
-----------------------
Weezer: Washington DC Demo
-----------------------
Prince & The New Power Generation: Sexy MF
-----------------------
Coldplay: X&Y
-----------------------
Prince & The New Power Generation: 7
-----------------------
Prince Paul: Itstrumental
-----------------------
Prince & The New Power Generation: 7
-----------------------
Prince & The New Power Generation: The Slaughterhouse
----------

Will Smith: Miami
-----------------------
Darude: Feel the Beat
-----------------------
Bonnie "Prince" Billy: The Letting Go
-----------------------
Prince & The New Power Generation: Cream
-----------------------
Prince Far I: Psalms for I
-----------------------
Darude: Sandstorm
-----------------------
Beyoncé: B'Day
-----------------------
Beyoncé: B'Day
-----------------------
Will Smith: Gettin’ Jiggy Wit It
-----------------------
Will Smith: Freakin' It
-----------------------
Will Smith: Freakin' It
-----------------------
Eminem: Lose Yourself
-----------------------
Bad Meets Evil: Bad Meets Evil: Scary Music: Renegades
-----------------------
Beyoncé: B'Day
-----------------------
Prince: 1999
-----------------------
Leo Sayer: Cool Touch
-----------------------
Leo Sayer: World Radio
-----------------------
Leo Sayer: Silverbird
-----------------------
Leo Sayer: The Collection
-----------------------
DJ Green Lantern feat. Eminem: Alive on Arrival
-----------------------

José Feliciano: Feliz Navidad
-----------------------
Weezer: El Scorcho
-----------------------
Darude: Feel the Beat / Sandstorm
-----------------------
U2: Zooropa
-----------------------
Beyoncé: Dangerously in Love
-----------------------
U2: No Line on the Horizon
-----------------------
Leo Sayer: Living In a Fantasy
-----------------------
Will Smith: Just the Two of Us
-----------------------
The Eagles: Bristol Express / Johnny's Tune
-----------------------
King Jammy: Selectors Choice, Volume 3
-----------------------
Eminem feat. Dr. Dre & 50 Cent: Crack a Bottle
-----------------------
Otis Rush: Groaning the Blues / If You Were Mine
-----------------------
Prince: 1999
-----------------------
Bonnie "Prince" Billy: Beware
-----------------------
Will Smith: Men in Black
-----------------------
U2: No Line On the Horizon
-----------------------
U2: No Line on the Horizon
-----------------------
Leo Sayer: Just a Boy
-----------------------
Leo Sayer: Another Year
--------

...

Prince Fatty: Survival of the Fattest
-----------------------
DJ Jazzy Jeff & The Fresh Prince: Platinum & Gold Collection
-----------------------
King Jammy: King Jammy's: Selector's Choice Vol. 3
-----------------------
Kay One: J.G.U.D.Z.S. [Jung genug um drauf zu scheissen]
-----------------------
Kay One: Prince Of Belvedair - Premium
-----------------------
Idina Menzel: Live: Barefoot At The Symphony
-----------------------
Idina Menzel: Idina Menzel
-----------------------
Idina Menzel: Let It Go Remixes (From "Frozen")
-----------------------
Idina Menzel: Let It Go Remixes (From "Frozen")
-----------------------
Idina Menzel: Gorgeous [Gabriel Diggs' Perfect 10 Remix]
-----------------------
Idina Menzel: Gorgeous [Craig C's Master Mix]
-----------------------
Idina Menzel: Gorgeous [Funky Junction & Antony Reale Remix]
-----------------------
Idina Menzel: Gorgeous [Lior Magal Vocal Dub]
-----------------------
Idina Menzel: Gorgeous [Scotty K Vocal Club]
-------------------

Kelly Price: One Family
-----------------------
King Jammy: Selectors Choice Volume 2
-----------------------
Original Motion Picture Soundtrack: Dirty Dancing
-----------------------
King Jammy: King At The Controls
-----------------------
King Jammy: Selectors Choice Volume 1
-----------------------
King Jammy: King Jammy's: Selector's Choice Vol. 1
-----------------------
King Jammy: King Jammy's: Selector's Choice Vol. 2
-----------------------
Darude: I Ran (So Far Away)
-----------------------
Darude: Label This! - US Special Edition
-----------------------
Darude: Label This!
-----------------------
Darude: Label This!
-----------------------
Darude: Feel the Beat
-----------------------
Darude: Sandstorm -- The Remixes
-----------------------
Darude: Rush
-----------------------
Darude: Rush
-----------------------
Darude: Rush
-----------------------
Darude: Rush
-----------------------
Darude: Before The Storm
-----------------------
Darude: I Ran (So Far Away) [Radio Edit]
-

Oscar Prince: Te Amo
-----------------------
D'Prince: Worldwide
-----------------------
Kay One: Lookalikes
-----------------------
Weezer: Make Believe
-----------------------
Kay One: Der Junge von damals
-----------------------
Weezer: Raditude
-----------------------
Idina Menzel: I See You
-----------------------
Prince Fox: Oxygen
-----------------------
Idina Menzel: idina.
-----------------------
Eminem: Relapse
-----------------------
Kay One: Mile High Club
-----------------------
Aerosmith: Just Push Play
-----------------------
Kay One: Der Junge von damals
-----------------------
Idina Menzel: Queen of Swords
-----------------------
Metallica: Hardwired…To Self-Destruct
-----------------------
Prince Alla: Prince Alla: The Best Of (Redemption Sounds present)
-----------------------
Kay One: Believe
-----------------------
Kay One: Der Junge von damals
-----------------------
Kay One: Der Junge von damals
-----------------------
Prince Daddy & the Hyena: I Thought You Didn

Sound Rush: Mystical Expedition
-----------------------
Tom Rush: No Tears Goodbye (The Schaefer Music Festival, Central Park 1972) [Live]
-----------------------
Prince Paul: Politics of the Business
-----------------------
Foster the People: Sacred Hearts Club
-----------------------
Tito Prince: Poétic justice
-----------------------
Coldplay: All I Can Think About Is You
-----------------------
Coldplay: Kaleidoscope
-----------------------
Prince: Purple Rain Deluxe (Expanded Edition)
-----------------------
Coldplay feat. The Chainsmokers: Something Just Like This (Tokyo Remix)
-----------------------
Haim: Little of Your Love
-----------------------
Ice Prince: Successful
-----------------------
Cyhi The Prynce: Movin' Around
-----------------------
カラオケうたプリンス: J-POP HIT SONG 9 (カラオケ)
-----------------------
Dylan Scott: Dylan Scott (Deluxe Edition)
-----------------------
Dennis Lloyd: Nevermind
-----------------------
Old Dominion: Happy Endings
-----------------------
Haim: S

Kygo & Miguel: Remind Me to Forget
-----------------------
Ice Prince: If I Tell You
-----------------------
Eminem: Nowhere Fast (Extended Version)
-----------------------
Panic! At the Disco: Pray For The Wicked
-----------------------
Abba Karib feat. Navid Zardi: Amawet
-----------------------
Prince Buster: Prince Buster Record Shack Presents: The Original Golden Oldies, Vol. 1
-----------------------
Cardi B: Invasion of Privacy
-----------------------
Calvin Harris feat. Dua Lipa: One Kiss
-----------------------
Keith Urban: Graffiti U
-----------------------
Drake: Nice For What
-----------------------
Jake Owen: I Was Jack (You Were Diane)
-----------------------
Nio Garcia feat. Casper Magico & Bad Bunny: Te Boté (Remix)
-----------------------
Carrie Underwood: Cry Pretty
-----------------------
Bazzi: COSMIC
-----------------------
5 Seconds of Summer: Youngblood (Deluxe)
-----------------------
Sugarland: Bigger
-----------------------
Bebe Rexha: Expectations
-----------

In [51]:
genre_df = pd.DataFrame.from_dict(genre_dic).T
genre_df.rename(index=str, columns={"index":"album_id"})
genre_df.drop_duplicates(subset='album',keep='first', inplace=True)
genre_df.reset_index(inplace=True)

In [66]:
genre_df.drop("level_0", axis=1, inplace=True)

In [55]:
from bs4 import BeautifulSoup
from splinter import Browser
from selenium import webdriver
import requests

In [80]:
genres = {}
total = len(genre_df['artist'])
for i in range(total):
    artist = genre_df['artist'][i]
    album = genre_df['album'][i]
    index = genre_df['index'][i]
    
    web = 'https://www.allmusic.com/search/albums/'
    
    #Set the header to trick the webpage security
    headers = requests.utils.default_headers()
    headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
    })
    
    #Create a splinter of the webpage to start scraping
    #driver = webdriver.Chrome()
    #executable_path = {"executable_path": "resources/chromedriver.exe"}
    #browser = Browser('chrome', **executable_path, headless=False)
    
    url = web + artist + " " + album
    
    #browser.visit(url)
    
    site = requests.get(url, headers=headers)
    soup = BeautifulSoup(site.content, 'html.parser')
    try:
        genre = soup.find("div", class_='genres').get_text()
        genre_strip = genre.replace("\n", "").strip()
        print(f"{artist}'s album, {album}, genre is {genre_strip}")
        print(f' Completed record {i} of {total}')
    except AttributeError:
        pass
    i+=1
    genres[f'{index}']={'genre':genre_strip}

Will Smith's album, Big Willie Style, genre is Rap
 Completed record 0 of 1706
U2's album, War, genre is Pop/Rock
 Completed record 1 of 1706
Led Zeppelin's album, Led Zeppelin, genre is Pop/Rock
 Completed record 2 of 1706
Rush's album, Power Windows, genre is Pop/Rock
 Completed record 3 of 1706
Will Smith's album, Willennium, genre is Rap
 Completed record 4 of 1706
Coldplay's album, Parachutes, genre is Pop/Rock
 Completed record 5 of 1706
Rush's album, Hold Your Fire, genre is Pop/Rock
 Completed record 6 of 1706
Rush's album, Counterparts, genre is Pop/Rock
 Completed record 7 of 1706
Otis Rush's album, 1956-1958: Cobra Recordings, genre is Blues
 Completed record 8 of 1706
Otis Rush's album, The Cobra Sessions 1956-1958, genre is Blues
 Completed record 9 of 1706
Slayer's album, Hell Awaits, genre is Pop/Rock
 Completed record 10 of 1706
Slayer's album, Show No Mercy, genre is Pop/Rock
 Completed record 11 of 1706
Metallica's album, ...And Justice for All, genre is Pop/Rock
 Com

Prince's album, Lovesexy, genre is R&B;
 Completed record 102 of 1706
Will Oldham's album, Guarapero: Lost Blues 2, genre is Pop/Rock
 Completed record 103 of 1706
Will Oldham's album, Joya, genre is Pop/Rock
 Completed record 104 of 1706
Darude's album, Feel the Beat, genre is Electronic
 Completed record 105 of 1706
A*Teens's album, Pop ’Til You Drop!, genre is Pop/Rock
 Completed record 106 of 1706
Will Smith's album, Greatest Hits, genre is Rap
 Completed record 107 of 1706
Merrilee Rush's album, Angel of the Morning, genre is Pop/Rock
 Completed record 109 of 1706
A*Teens's album, New Arrival, genre is Pop/Rock
 Completed record 110 of 1706
Eagles's album, Desperado, genre is Easy Listening
 Completed record 111 of 1706
Bonnie "Prince" Billy's album, Blue Lotus Feet, genre is Pop/Rock
 Completed record 112 of 1706
Darude's album, Ignition, genre is Pop/Rock
 Completed record 114 of 1706
Kelly Price's album, Priceless, genre is R&B;
 Completed record 115 of 1706
Prince Alla's album

Otis Rush's album, A Fool for You, genre is Pop/Rock
 Completed record 215 of 1706
Otis Rush's album, Cold Day in Hell, genre is Blues
 Completed record 216 of 1706
Jennifer Rush's album, Out of My Hands, genre is Pop/Rock
 Completed record 217 of 1706
Frank Marino & Mahogany Rush's album, Eye of the Storm, genre is Pop/Rock
 Completed record 218 of 1706
Frank Marino & Mahogany Rush's album, What's Next, genre is Pop/Rock
 Completed record 219 of 1706
Frank Marino & Mahogany Rush's album, World Anthem, genre is Pop/Rock
 Completed record 220 of 1706
The Real ABBA Gold's album, Janus, genre is Pop/Rock
 Completed record 221 of 1706
Frank Marino & Mahogany Rush's album, Maxoom, genre is Pop/Rock
 Completed record 224 of 1706
U2's album, How to Dismantle an Atomic Bomb, genre is Pop/Rock
 Completed record 227 of 1706
Jennifer Rush's album, Wings of Desire, genre is Pop/Rock
 Completed record 228 of 1706
Prince's album, The Work, Volume 1, genre is Pop/Rock
 Completed record 229 of 1706
Pr

Leo Sayer's album, Voice in My Head, genre is Pop/Rock
 Completed record 343 of 1706
Eagles of Death Metal's album, I Want You So Hard (Boys Bad News), genre is Pop/Rock
 Completed record 344 of 1706
Bonnie "Prince" Billy's album, The Letting Go, genre is Pop/Rock
 Completed record 345 of 1706
Prince & The New Power Generation's album, Cream, genre is R&B;
 Completed record 346 of 1706
Beyoncé's album, B'Day, genre is R&B;
 Completed record 347 of 1706
Will Smith's album, Gettin’ Jiggy Wit It, genre is Rap, R&B;
 Completed record 348 of 1706
Leo Sayer's album, World Radio, genre is Pop/Rock
 Completed record 351 of 1706
Leo Sayer's album, The Collection, genre is Pop/Rock
 Completed record 352 of 1706
Kelly Price's album, This Is Who I Am, genre is Pop/Rock
 Completed record 356 of 1706
Bonnie "Prince" Billy's album, Cold & Wet, genre is Pop/Rock
 Completed record 357 of 1706
Otis Rush's album, The Essential Otis Rush: The Classic Cobra Recordings 1956-1958, genre is Blues
 Completed r

Jennifer Rush's album, Destiny, genre is Pop/Rock
 Completed record 459 of 1706
Eminem's album, Relapse: Refill, genre is Rap
 Completed record 460 of 1706
Tom Rush's album, Live at Symphony Hall, Boston, genre is Folk
 Completed record 461 of 1706
Otis Rush's album, Otis Rush: Live at Montreux 1986, genre is Blues
 Completed record 465 of 1706
Leo Sayer's album, Have You Ever Been In Love, genre is Pop/Rock
 Completed record 466 of 1706
Adele Bertei's album, Little Lives, genre is Pop/Rock, Electronic
 Completed record 467 of 1706
Abba Revival Band's album, Thank You for the Music, genre is Pop/Rock
 Completed record 469 of 1706
Pantha du Prince's album, Black Noise, genre is Electronic
 Completed record 470 of 1706
94 East's album, Minneapolis Genius (The Historic 1977 Recordings), genre is R&B;, Blues, Pop/Rock
 Completed record 471 of 1706
Rush's album, The Big Money, genre is Pop/Rock
 Completed record 472 of 1706
Jennifer Rush's album, Now Is the Hour, genre is Electronic, Pop/Ro

...

Rush of Fools's album, Premiere Performance Plus: Wonder of the World, genre is Religious
 Completed record 981 of 1706
Rush of Fools's album, Wonder of the World, genre is Religious
 Completed record 982 of 1706
カラオケうたプリンス's album, アニソン特集 Vol.1(カラオケ), genre is International
 Completed record 983 of 1706
The Eagles's album, Smash Hits from the Eagles, genre is Pop/Rock
 Completed record 987 of 1706
Eagles Gift's album, Eagles Gift, genre is Children's
 Completed record 988 of 1706
Haim Ifargan חיים איפרגן's album, פרק אחר, genre is Jazz
 Completed record 989 of 1706
Starlito's album, Eyes Closed, genre is Pop/Rock
 Completed record 990 of 1706
Prince Paul's album, Prince Among Thieves, genre is Rap
 Completed record 995 of 1706
Prince Paul's album, Psychoanalysis: What Is It?!, genre is Rap
 Completed record 996 of 1706
Rush of Fools's album, Lay Me Down, genre is Blues, Pop/Rock
 Completed record 997 of 1706
Haim's album, If I Could Change Your Mind, genre is Pop/Rock
 Completed recor

Prince Royce's album, Culpa al Corazón, genre is Latin
 Completed record 1170 of 1706
Adele's album, Why do you love me, genre is Pop/Rock, R&B;
 Completed record 1173 of 1706
Prince Royce's album, Extraordinary, genre is Latin
 Completed record 1175 of 1706
Eagles & Butterflies's album, We Get High, genre is Electronic
 Completed record 1176 of 1706
Prince Fox's album, Musings, genre is Electronic
 Completed record 1177 of 1706
Prince Fox's album, I Don't Wanna Love You, genre is R&B;
 Completed record 1178 of 1706
Coldplay's album, Everglow, genre is Pop/Rock
 Completed record 1179 of 1706
Prince Royce's album, Back It Up, genre is International
 Completed record 1181 of 1706
Prince Royce's album, Handcuffs, genre is Latin
 Completed record 1182 of 1706
Starlito's album, I'm Moving to Houston, genre is Rap
 Completed record 1183 of 1706
Adele's album, 21' Sampler, genre is Electronic
 Completed record 1184 of 1706
Adele's album, Rolling In The Deep, genre is Pop/Rock, R&B;
 Completed

Arabian Prince's album, Innovative Life Anthology 1984-1989, genre is Rap
 Completed record 1310 of 1706
Microwave Prince's album, I Need Your Love, genre is Pop/Rock
 Completed record 1311 of 1706
Vitamin String Quartet's album, VSQ Performs the Hits of 2013 Vol. 2, genre is Pop/Rock
 Completed record 1315 of 1706
Kelly Price's album, One Family, genre is R&B;, Holiday
 Completed record 1316 of 1706
King Jammy's album, Selectors Choice Volume 2, genre is Reggae
 Completed record 1317 of 1706
Original Motion Picture Soundtrack's album, Dirty Dancing, genre is Pop/Rock, Stage & Screen, R&B;
 Completed record 1318 of 1706
King Jammy's album, King At The Controls, genre is Reggae
 Completed record 1319 of 1706
King Jammy's album, Selectors Choice Volume 1, genre is Reggae
 Completed record 1320 of 1706
King Jammy's album, King Jammy's: Selector's Choice Vol. 1, genre is Reggae
 Completed record 1321 of 1706
King Jammy's album, King Jammy's: Selector's Choice Vol. 2, genre is Reggae
 Compl

Prince's album, Purple Rain, genre is R&B;
 Completed record 1467 of 1706
MercyMe's album, Lifer, genre is Religious
 Completed record 1468 of 1706
The Chainsmokers's album, Memories...Do Not Open, genre is Pop/Rock, Electronic
 Completed record 1470 of 1706
Prince Royce's album, Phase 2, genre is Latin
 Completed record 1471 of 1706
Eden Prince's album, Señorita, genre is Pop/Rock
 Completed record 1472 of 1706
Eartha Kitt's album, Revisited, genre is Vocal
 Completed record 1473 of 1706
Bonnie "Prince" Billy's album, Best Troubador, genre is Pop/Rock
 Completed record 1474 of 1706
カラオケうたプリンス's album, J-POP HIT SONG 4 (カラオケ), genre is Stage & Screen
 Completed record 1475 of 1706
Sound Rush's album, Back To The Roots, genre is Pop/Rock, Blues
 Completed record 1476 of 1706
Weezer's album, Feels Like Summer, genre is Pop/Rock
 Completed record 1477 of 1706
Prince Buster's album, Judge Dread, genre is Reggae
 Completed record 1480 of 1706
Chris Stapleton's album, From A Room: Volume 1, 

Oscar Prince's album, Me Duele en el Alma, genre is Pop/Rock
 Completed record 1602 of 1706
XXXTENTACION's album, ?, genre is Rap
 Completed record 1604 of 1706
AJR's album, Burn the House Down, genre is Pop/Rock
 Completed record 1606 of 1706
Jason Aldean's album, Rearview Town, genre is Country
 Completed record 1608 of 1706
Kygo & Miguel's album, Remind Me to Forget, genre is Pop/Rock
 Completed record 1609 of 1706
Ice Prince's album, If I Tell You, genre is New Age
 Completed record 1610 of 1706
Panic! At the Disco's album, Pray For The Wicked, genre is Pop/Rock
 Completed record 1612 of 1706
Cardi B's album, Invasion of Privacy, genre is Rap
 Completed record 1614 of 1706
Keith Urban's album, Graffiti U, genre is Country
 Completed record 1616 of 1706
Drake's album, Nice For What, genre is Rap
 Completed record 1617 of 1706
Jake Owen's album, I Was Jack (You Were Diane), genre is Country
 Completed record 1618 of 1706
Carrie Underwood's album, Cry Pretty, genre is Country
 Complet

In [141]:
gen_df = pd.DataFrame(genres).T
genres_df = genre_df.merge(gen_df, left_index=True, right_index=True)
genres_df.drop("genre_x",axis=1, inplace=True)
genres_df.rename(columns={"genre_y":"genre"}, inplace=True)

In [144]:
for i in range(len(genres_df['genre'])):
    text = genres_df['genre'][i]
    text_new = text.split(',',1)[0]
    genres_df['genre'][i] = text_new

In [148]:
genres_df.replace("R&B;","R&B")

,album,artist,genre
index,,,
10266031,Big Willie Style,Will Smith,Rap
10266041,War,U2,Pop/Rock
10266127,Led Zeppelin,Led Zeppelin,Pop/Rock
10266180,Power Windows,Rush,Pop/Rock
10266202,Willennium,Will Smith,Rap
10266231,Parachutes,Coldplay,Pop/Rock
10266299,Hold Your Fire,Rush,Pop/Rock
10266444,Counterparts,Rush,Pop/Rock
10267122,1956-1958: Cobra Recordings,Otis Rush,Blues


In [149]:
#genres_df.to_csv('genres_for_unique.csv',encoding='utf-8')

In [13]:
genres_df = pd.read_csv('genres_for_unique.csv')
genres_df.reset_index(inplace=True)

In [14]:
genres_df.drop("level_0", axis=1,inplace=True)

In [16]:
genres_df.rename(columns={"index":"album_id"},inplace=True)
genres_df.head()

,album_id,album,artist,genre
0,10266031,Big Willie Style,Will Smith,Rap
1,10266041,War,U2,Pop/Rock
2,10266127,Led Zeppelin,Led Zeppelin,Pop/Rock
3,10266180,Power Windows,Rush,Pop/Rock
4,10266202,Willennium,Will Smith,Rap


In [8]:
genres_df.to_csv('cleaned_genre.csv',encoding='utf-8')

NameError: name 'genres_df' is not defined

In [8]:
genres_df = pd.read_csv('cleaned_genre.csv')

In [9]:
genres_df['album_id'][0]

10266031

In [9]:
table = sqlalchemy.Table('genres', meta, autoload = True)
x=0
for x in range(len(genres_df['album_id'])):
    package = {
        'album_id': str(genres_df['album_id'][x]),
        'album_name': genres_df['album_name'][x],
        'artist': genres_df['artist'][x],
        'genre': genres_df['genre'][x]
    }
    try:
        print("writing package to database")
        session.execute(table.insert(), package)
    except:
        next
    print("committing to database")
    session.commit()
    x +=1

writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
com

writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
com

writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
com

writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
com

writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
com

...

writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
com

writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
com

writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
com

writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
com

writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
committing to database
writing package to database
com

In [4]:
#Drop Table if the commit doesnt work
sql = text('DROP TABLE IF EXISTS genres;')
result = engine.execute(sql)